In [ ]:
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set page config
st.set_page_config(page_title="4-Tank System LSTM Forecasting", layout="wide")

st.title("🏭 Neural Network Forecasting: 4-Tank System")
st.markdown("### Demonstrating LSTM for Chemical Process Control")

# Sidebar for parameters
st.sidebar.header("Model Parameters")
sequence_length = st.sidebar.slider("Sequence Length (time steps)", 10, 100, 30)
prediction_horizon = st.sidebar.slider("Prediction Horizon (steps)", 1, 50, 10)
epochs = st.sidebar.slider("Training Epochs", 10, 100, 50)

# Load or generate data
@st.cache_data
def load_tank_data():
    """Load the 4-tank system data"""
    try:
        # Try to load existing data
        levels = pd.read_csv('tank_levels_2.csv')
        inputs = pd.read_csv('inputs_2.csv')
        
        # Combine data
        data = pd.merge(levels, inputs, on='Time')
        return data
    except:
        st.error("Data files not found. Please run the 4-tank simulation first to generate tank_levels_2.csv and inputs_2.csv")
        return None

def prepare_lstm_data(data, sequence_length, prediction_horizon):
    """Prepare data for LSTM training"""
    # Features: current levels + inputs
    features = ['Tank1', 'Tank2', 'Tank3', 'Tank4', 'v1', 'v2']
    targets = ['Tank1', 'Tank2', 'Tank3', 'Tank4']
    
    # Normalize data
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    
    X_scaled = scaler_X.fit_transform(data[features])
    y_scaled = scaler_y.fit_transform(data[targets])
    
    # Create sequences
    X, y = [], []
    for i in range(len(data) - sequence_length - prediction_horizon + 1):
        X.append(X_scaled[i:(i + sequence_length)])
        y.append(y_scaled[i + sequence_length:i + sequence_length + prediction_horizon])
    
    return np.array(X), np.array(y), scaler_X, scaler_y

def build_lstm_model(input_shape, output_shape):
    """Build LSTM model architecture"""
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(np.prod(output_shape), activation='linear')
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Main app
def main():
    data = load_tank_data()
    if data is None:
        return
    
    # Display data overview
    col1, col2 = st.columns([2, 1])
    
    with col1:
        st.subheader("📊 System Data Overview")
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=['Tank Levels', 'Input Voltages'],
            vertical_spacing=0.1
        )
        
        # Tank levels
        fig.add_trace(go.Scatter(x=data['Time'], y=data['Tank1'], name='Tank 1', line=dict(color='blue')), row=1, col=1)
        fig.add_trace(go.Scatter(x=data['Time'], y=data['Tank2'], name='Tank 2', line=dict(color='green')), row=1, col=1)
        fig.add_trace(go.Scatter(x=data['Time'], y=data['Tank3'], name='Tank 3', line=dict(color='red')), row=1, col=1)
        fig.add_trace(go.Scatter(x=data['Time'], y=data['Tank4'], name='Tank 4', line=dict(color='cyan')), row=1, col=1)
        
        # Input voltages
        fig.add_trace(go.Scatter(x=data['Time'], y=data['v1'], name='v1', line=dict(color='orange')), row=2, col=1)
        fig.add_trace(go.Scatter(x=data['Time'], y=data['v2'], name='v2', line=dict(color='purple')), row=2, col=1)
        
        fig.update_layout(height=600, title_text="4-Tank System Dynamics")
        fig.update_xaxes(title_text="Time (s)", row=2, col=1)
        fig.update_yaxes(title_text="Height (m)", row=1, col=1)
        fig.update_yaxes(title_text="Voltage (V)", row=2, col=1)
        
        st.plotly_chart(fig, use_container_width=True)
    
    with col2:
        st.subheader("📈 Data Statistics")
        st.write(data.describe())
    
    # Train/Test split
    split_ratio = st.sidebar.slider("Train/Test Split", 0.6, 0.9, 0.8)
    split_idx = int(len(data) * split_ratio)
    
    train_data = data[:split_idx]
    test_data = data[split_idx:]
    
    st.subheader("🤖 LSTM Model Training")
    
    if st.button("🚀 Train LSTM Model"):
        with st.spinner("Training LSTM model..."):
            # Prepare data
            X_train, y_train, scaler_X, scaler_y = prepare_lstm_data(train_data, sequence_length, prediction_horizon)
            X_test, y_test, _, _ = prepare_lstm_data(test_data, sequence_length, prediction_horizon)
            
            # Build model
            model = build_lstm_model((sequence_length, 6), (prediction_horizon, 4))
            
            # Train model
            history = model.fit(
                X_train, y_train.reshape(y_train.shape[0], -1),
                epochs=epochs,
                batch_size=32,
                validation_split=0.2,
                verbose=0
            )
            
            # Make predictions
            y_pred = model.predict(X_test)
            y_pred = y_pred.reshape(y_pred.shape[0], prediction_horizon, 4)
            
            # Inverse transform
            y_test_orig = scaler_y.inverse_transform(y_test.reshape(-1, 4)).reshape(y_test.shape)
            y_pred_orig = scaler_y.inverse_transform(y_pred.reshape(-1, 4)).reshape(y_pred.shape)
            
            # Calculate metrics
            mse = mean_squared_error(y_test_orig.flatten(), y_pred_orig.flatten())
            mae = mean_absolute_error(y_test_orig.flatten(), y_pred_orig.flatten())
            
            # Display results
            col1, col2, col3 = st.columns(3)
            with col1:
                st.metric("MSE", f"{mse:.6f}")
            with col2:
                st.metric("MAE", f"{mae:.6f}")
            with col3:
                st.metric("RMSE", f"{np.sqrt(mse):.6f}")
            
            # Plot training history
            fig_history = go.Figure()
            fig_history.add_trace(go.Scatter(y=history.history['loss'], name='Training Loss'))
            fig_history.add_trace(go.Scatter(y=history.history['val_loss'], name='Validation Loss'))
            fig_history.update_layout(title="Training History", xaxis_title="Epoch", yaxis_title="Loss")
            st.plotly_chart(fig_history, use_container_width=True)
            
            # Plot predictions vs actual
            st.subheader("🎯 Predictions vs Actual")
            
            # Show predictions for first few test samples
            n_samples = min(5, len(y_test_orig))
            
            for tank in range(4):
                fig_pred = go.Figure()
                
                for i in range(n_samples):
                    time_steps = np.arange(prediction_horizon)
                    fig_pred.add_trace(go.Scatter(
                        x=time_steps, 
                        y=y_test_orig[i, :, tank], 
                        name=f'Actual Sample {i+1}',
                        line=dict(dash='solid')
                    ))
                    fig_pred.add_trace(go.Scatter(
                        x=time_steps, 
                        y=y_pred_orig[i, :, tank], 
                        name=f'Predicted Sample {i+1}',
                        line=dict(dash='dash')
                    ))
                
                fig_pred.update_layout(
                    title=f"Tank {tank+1} Predictions",
                    xaxis_title="Time Steps",
                    yaxis_title="Height (m)"
                )
                st.plotly_chart(fig_pred, use_container_width=True)
    
    # Interactive prediction
    st.subheader("🎮 Interactive Prediction")
    st.markdown("Adjust current conditions and see LSTM predictions:")
    
    col1, col2 = st.columns(2)
    with col1:
        h1_current = st.slider("Current Tank 1 Level (m)", 0.0, 1.0, 0.3)
        h2_current = st.slider("Current Tank 2 Level (m)", 0.0, 1.0, 0.3)
        h3_current = st.slider("Current Tank 3 Level (m)", 0.0, 1.0, 0.2)
        h4_current = st.slider("Current Tank 4 Level (m)", 0.0, 1.0, 0.2)
    
    with col2:
        v1_current = st.slider("Current v1 (V)", 0.0, 10.0, 5.0)
        v2_current = st.slider("Current v2 (V)", 0.0, 10.0, 5.0)
        
    st.info("💡 **Learning Objectives:**")
    st.markdown("""
    - **Time Series Forecasting**: Predict future tank levels based on current state
    - **Multivariate Input**: Use both tank levels and control inputs
    - **Process Control**: Understand how inputs affect system dynamics
    - **Model Validation**: Compare predictions with actual process behavior
    - **Hyperparameter Tuning**: Adjust sequence length, prediction horizon, epochs
    """)

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
import pickle
import os

# ===== TRAINING THE MODEL =====
# Load data
df = pd.read_csv("inputs_2.csv", sep=',')
df = df.drop('Time', axis=1)

# Define input and output columns
input_cols = ['v1', 'v2']
output_cols = ['Tank1', 'Tank2', 'Tank3', 'Tank4']

# Extract values
X_raw = df[input_cols].values
y_raw = df[output_cols].values

# Scale inputs and outputs separately
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X_scaled = x_scaler.fit_transform(X_raw)
y_scaled = y_scaler.fit_transform(y_raw)

# Convert to LSTM sequences
def create_sequences(X, y, window=10):
    X_seq, y_seq = [], []
    for i in range(window, len(X)):
        X_seq.append(X[i-window:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

window = 10
X_seq, y_seq = create_sequences(X_scaled, y_scaled, window)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

# Build LSTM model
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(4))  # Output for 4 tanks
model.compile(optimizer='adam', loss='mse')

# Train model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

# ===== SAVING THE MODEL AND COMPONENTS =====

print("Saving model and components...")

# 1. Save the Keras model (RECOMMENDED)
model.save('lstm_4tank_model.h5')  # Saves architecture + weights + optimizer state
# OR for newer format:
# model.save('lstm_4tank_model.keras')

# 2. Save the scalers using joblib (THIS IS CORRECT)
joblib.dump(x_scaler, 'x_scaler.pkl')
joblib.dump(y_scaler, 'y_scaler.pkl')

# 3. Save model metadata and parameters
model_metadata = {
    'window_size': window,
    'input_cols': input_cols,
    'output_cols': output_cols,
    'model_architecture': {
        'lstm_units': [64, 64],
        'dropout_rate': 0.2,
        'dense_units': 4
    },
    'training_params': {
        'epochs': 50,
        'batch_size': 32,
        'validation_split': 0.1
    },
    'input_shape': X_train.shape,
    'output_shape': y_train.shape
}

# Save metadata
joblib.dump(model_metadata, 'model_metadata.pkl')

# 4. Optional: Save training history
joblib.dump(history.history, 'training_history.pkl')

print("✅ Model saved successfully!")
print("Files created:")
print("- lstm_4tank_model.h5 (main model)")
print("- x_scaler.pkl (input scaler)")
print("- y_scaler.pkl (output scaler)")
print("- model_metadata.pkl (parameters)")
print("- training_history.pkl (training history)")

# ===== LOADING AND USING THE SAVED MODEL =====

def load_trained_model():
    """Load the complete trained model with all components"""
    
    # Load the Keras model
    loaded_model = load_model('lstm_4tank_model.h5')
    
    # Load the scalers
    loaded_x_scaler = joblib.load('x_scaler.pkl')
    loaded_y_scaler = joblib.load('y_scaler.pkl')
    
    # Load metadata
    loaded_metadata = joblib.load('model_metadata.pkl')
    
    print("✅ Model loaded successfully!")
    return loaded_model, loaded_x_scaler, loaded_y_scaler, loaded_metadata

def predict_tank_levels(model, x_scaler, y_scaler, metadata, input_sequence):
    """
    Make predictions using the loaded model
    
    Args:
        model: Loaded Keras model
        x_scaler: Loaded input scaler
        y_scaler: Loaded output scaler
        metadata: Model metadata
        input_sequence: New input data [v1, v2] for last 'window_size' time steps
    
    Returns:
        Predicted tank levels [Tank1, Tank2, Tank3, Tank4]
    """
    
    # Scale the input
    input_scaled = x_scaler.transform(input_sequence)
    
    # Reshape for LSTM (add batch dimension)
    input_reshaped = input_scaled.reshape(1, metadata['window_size'], len(metadata['input_cols']))
    
    # Make prediction
    prediction_scaled = model.predict(input_reshaped, verbose=0)
    
    # Inverse transform to get actual tank levels
    prediction = y_scaler.inverse_transform(prediction_scaled)
    
    return prediction[0]  # Remove batch dimension

# Example usage of the loaded model
if __name__ == "__main__":
    # Test loading
    try:
        loaded_model, loaded_x_scaler, loaded_y_scaler, loaded_metadata = load_trained_model()
        
        # Example prediction with new data
        # Create example input sequence (last 10 time steps of v1, v2)
        example_input = np.array([
            [5.0, 3.0],  # v1=5V, v2=3V
            [5.1, 3.1],
            [5.2, 3.0],
            [5.0, 2.9],
            [4.9, 3.0],
            [5.0, 3.1],
            [5.1, 3.0],
            [5.0, 3.0],
            [5.0, 3.0],
            [5.0, 3.0]   # Most recent values
        ])
        
        # Make prediction
        predicted_levels = predict_tank_levels(
            loaded_model, loaded_x_scaler, loaded_y_scaler, 
            loaded_metadata, example_input
        )
        
        print(f"\nPredicted tank levels:")
        for i, tank in enumerate(loaded_metadata['output_cols']):
            print(f"{tank}: {predicted_levels[i]:.4f} m")
            
    except FileNotFoundError as e:
        print(f"❌ Model files not found: {e}")
        print("Please run the training code first to save the model.")

# ===== FOR STREAMLIT DEPLOYMENT =====

def create_streamlit_model_loader():
    """
    Create a function specifically for loading in Streamlit
    """
    
    @st.cache_resource
    def load_model_for_streamlit():
        return load_trained_model()
    
    return load_model_for_streamlit

Epoch 1/50
180/180 [==============================] - 4s 10ms/step - loss: 0.0433 - val_loss: 0.0214
Epoch 2/50
180/180 [==============================] - 1s 7ms/step - loss: 0.0343 - val_loss: 0.0212
Epoch 3/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0340 - val_loss: 0.0246
Epoch 4/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0334 - val_loss: 0.0230
Epoch 5/50
180/180 [==============================] - 1s 7ms/step - loss: 0.0325 - val_loss: 0.0201
Epoch 6/50
180/180 [==============================] - 1s 7ms/step - loss: 0.0322 - val_loss: 0.0196
Epoch 7/50
180/180 [==============================] - 1s 7ms/step - loss: 0.0317 - val_loss: 0.0192
Epoch 8/50
180/180 [==============================] - 1s 7ms/step - loss: 0.0310 - val_loss: 0.0192
Epoch 9/50
180/180 [==============================] - 1s 8ms/step - loss: 0.0302 - val_loss: 0.0286
Epoch 10/50
180/180 [==============================] - 1s 7ms/step - loss: 0.0293 - val_loss: 0.022

c:\Users\uqmkonar\Anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


✅ Model loaded successfully!

Predicted tank levels:
Tank1: 0.0442 m
Tank2: 0.1817 m
Tank3: 0.0703 m
Tank4: 0.1660 m
